In [1]:
import numpy as np
import torch
import torch.utils.data as data
import pandas as pd
from src.error_measures import get_accuracy, get_error_measures
from src.models import TwoLayerPerceptron, TwoLayerSynaptic, TwoLayerLeaky, ThreeLayerLeaky
from src.train_model import training_loop

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cpu')

In [3]:
# setting seeds
np.random.seed(1234)
torch.manual_seed(1234)

## Data

In [4]:
data06 = pd.read_csv("../data/processed/target06.csv")
data06.drop(columns=["weekday"], inplace=True)

In [5]:
all_data = data.TensorDataset(torch.from_numpy((data06.values[:,:-1] - data06.values[:,:-1].min(0)) / data06.values[:,:-1].ptp(0)).float(), torch.from_numpy(data06.values[:,-1]).float())  # with normalization
train_dataset, test_dataset, valid_dataset = torch.utils.data.random_split(all_data, (round(0.7 * len(all_data)), round(0.2 * len(all_data)), round(0.1 * len(all_data))))

In [6]:
train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=False)
valid_loader = data.DataLoader(valid_dataset, batch_size=256, shuffle=False, drop_last=False)

### Simple regressor

In [7]:
model1 = TwoLayerPerceptron(num_inputs=8, num_hidden=2500, num_outputs=1)
print(model1)

TwoLayerPerceptron(
  (lin1): Linear(in_features=8, out_features=2500, bias=True)
  (bn1): BatchNorm1d(2500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (lin2): Linear(in_features=2500, out_features=1, bias=True)
)


In [8]:
training_loop(model1, train_loader, valid_loader, device, num_epochs=25, validation=True)

  4%|▍         | 1/25 [00:00<00:10,  2.23it/s]

MAE: 6.849618503261853, MSE:87.49401621002367,
RMSE: 9.353823614438305, Index of Agreement: 0.8743003068535169


 16%|█▌        | 4/25 [00:01<00:09,  2.17it/s]

MAE: 6.4475028922792115, MSE:81.10340287910249,
RMSE: 9.005742772203883, Index of Agreement: 0.8881070828258529


 28%|██▊       | 7/25 [00:03<00:07,  2.37it/s]

...

MAE: 5.864843923094645, MSE:70.146789620956,
RMSE: 8.375368028985712, Index of Agreement: 0.8967486877969018


 88%|████████▊ | 22/25 [00:09<00:01,  2.23it/s]

MAE: 5.845930581544176, MSE:70.26235716050107,
RMSE: 8.382264441098304, Index of Agreement: 0.8988298622046791


100%|██████████| 25/25 [00:11<00:00,  2.22it/s]

MAE: 5.907317892802244, MSE:67.65397505074812,
RMSE: 8.225203647980281, Index of Agreement: 0.9007469178037413
Accuracy on validation dataset: 49.94%


In [9]:
print(f'Accuracy on test set: {get_accuracy(model1, test_loader, device=device, pct_close=0.25)*100}%')
get_error_measures(model1, test_loader, device=device, print_e=True)

Accuracy on test set: 49.36%
MAE: 6.290849704549491, MSE:75.09738563387732,
RMSE: 8.665874776032558, Index of Agreement: 0.8945206682864716


(6.290849704549491, 75.09738563387732, 8.665874776032558, 0.8945206682864716)

### SNN with Leaky layer

In [10]:
model2 = TwoLayerLeaky(num_inputs=8, num_hidden=2500, num_outputs=1)
print(model2)

TwoLayerLeaky(
  (fc1): Linear(in_features=8, out_features=2500, bias=True)
  (lif): Leaky()
  (fc2): Linear(in_features=2500, out_features=1, bias=True)
)


In [11]:
training_loop(model2, train_loader, valid_loader, device, num_epochs=25, validation=True)

  4%|▍         | 1/25 [00:03<01:12,  3.02s/it]

MAE: 10.102531865290823, MSE:180.81721470228982,
RMSE: 13.446829169075132, Index of Agreement: 0.4417148798000601


 16%|█▌        | 4/25 [00:11<01:03,  3.02s/it]

MAE: 7.377225428233946, MSE:98.1110851388681,
RMSE: 9.905103994348979, Index of Agreement: 0.8292479259576211


 28%|██▊       | 7/25 [00:20<00:52,  2.91s/it]

...

 88%|████████▊ | 22/25 [01:06<00:09,  3.33s/it]

MAE: 6.448959514860473, MSE:82.54102036020282,
RMSE: 9.085208878182318, Index of Agreement: 0.8803436512012981


100%|██████████| 25/25 [01:15<00:00,  3.02s/it]

MAE: 6.526208096432548, MSE:82.06711957267152,
RMSE: 9.059090438486168, Index of Agreement: 0.8761569532574729


Accuracy on validation dataset: 46.82%


In [12]:
print(f'Accuracy on test set: {get_accuracy(model2, test_loader, device=device, pct_close=0.2)*100}%')
get_error_measures(model2, test_loader, device=device, print_e=True)

Accuracy on test set: 36.3%
MAE: 6.953040434859392, MSE:92.97737715032781,
RMSE: 9.642477749537607, Index of Agreement: 0.8656423734240957


(6.953040434859392, 92.97737715032781, 9.642477749537607, 0.8656423734240957)

### SNN with Synaptic layer

In [13]:
model3 = TwoLayerSynaptic(num_inputs=8, num_hidden=2500, num_outputs=1)
print(model3)

TwoLayerSynaptic(
  (fc1): Linear(in_features=8, out_features=2500, bias=True)
  (lif): Synaptic()
  (fc2): Linear(in_features=2500, out_features=1, bias=True)
)


In [14]:
training_loop(model3, train_loader, valid_loader, device, num_epochs=25, validation=True)

  4%|▍         | 1/25 [00:14<05:54, 14.77s/it]

MAE: 9.75487863507574, MSE:172.99616797189483,
RMSE: 13.152800765308308, Index of Agreement: 0.4691464450403874


 16%|█▌        | 4/25 [00:54<04:51, 13.86s/it]

MAE: 9.468381516368403, MSE:153.15617643842742,
RMSE: 12.37562832499536, Index of Agreement: 0.6401932337676255


 28%|██▊       | 7/25 [01:39<04:29, 14.97s/it]

...

 88%|████████▊ | 22/25 [05:32<00:46, 15.65s/it]

MAE: 6.383086662071978, MSE:79.56384787203773,
RMSE: 8.919856942352704, Index of Agreement: 0.8763630087821215


100%|██████████| 25/25 [06:17<00:00, 15.08s/it]

MAE: 6.355549137027277, MSE:79.88520700213164,
RMSE: 8.937852482679027, Index of Agreement: 0.8779368677609976


Accuracy on validation dataset: 49.71%


In [15]:
print(f'Accuracy on test set: {get_accuracy(model3, test_loader, device=device, pct_close=0.25)*100}%')
get_error_measures(model3, test_loader, device=device, print_e=True)

Accuracy on test set: 48.84%
MAE: 6.745554911642405, MSE:90.66382020327872,
RMSE: 9.521755100992607, Index of Agreement: 0.8667303761374314


(6.745554911642405, 90.66382020327872, 9.521755100992607, 0.8667303761374314)

### Deep SNN

In [16]:
model4 = ThreeLayerLeaky(num_inputs=8, num_hidden=500, num_outputs=1)
print(model4)

ThreeLayerLeaky(
  (lin1): Linear(in_features=8, out_features=500, bias=True)
  (lif): Leaky()
  (lin2): Linear(in_features=500, out_features=500, bias=True)
  (bn1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (lin3): Linear(in_features=500, out_features=1, bias=True)
)


In [17]:
training_loop(model4, train_loader, valid_loader, device, num_epochs=25, validation=True)

  4%|▍         | 1/25 [00:05<02:09,  5.38s/it]

MAE: 8.494763001678995, MSE:146.39239797611285,
RMSE: 12.099272621778255, Index of Agreement: 0.7043853955084804


 16%|█▌        | 4/25 [00:20<01:48,  5.16s/it]

MAE: 7.742682053450215, MSE:120.37209132037495,
RMSE: 10.971421572447891, Index of Agreement: 0.770389735190173


 28%|██▊       | 7/25 [00:35<01:30,  5.03s/it]

...

 88%|████████▊ | 22/25 [01:49<00:14,  4.95s/it]

MAE: 7.502682121916314, MSE:103.54115032906788,
RMSE: 10.175517202042748, Index of Agreement: 0.8486850928960352


100%|██████████| 25/25 [02:03<00:00,  4.95s/it]

MAE: 7.09589591053869, MSE:99.68609117601463,
RMSE: 9.984292222086383, Index of Agreement: 0.8480214315638881


Accuracy on validation dataset: 43.580000000000005%


In [18]:
print(f'Accuracy on test set: {get_accuracy(model4, test_loader, device=device, pct_close=0.25)*100}%')
get_error_measures(model4, test_loader, device=device, print_e=True)

Accuracy on test set: 42.66%
MAE: 7.300635795786202, MSE:100.46806138715202,
RMSE: 10.023375748077692, Index of Agreement: 0.8553494401611195


(7.300635795786202, 100.46806138715202, 10.023375748077692, 0.8553494401611195)